In [1]:
import os
import time
import datetime
from pprint import pprint
from dotenv import load_dotenv

from prism_agent_open_api_specification_client import Client
from prism_agent_open_api_specification_client.models import ConnectionCollection,Connection,ConnectionInvitation,CreateConnectionRequest,AcceptConnectionInvitationRequest
from prism_agent_open_api_specification_client.api.connections_management import get_connections,get_connection,create_connection,accept_connection_invitation
from prism_agent_open_api_specification_client.types import Response
from prism_agent_open_api_specification_client.models import IssueCredentialRecord, CreateIssueCredentialRecordRequest, CreateIssueCredentialRecordRequestClaims, IssueCredentialRecordCollection
from prism_agent_open_api_specification_client.api.issue_credentials_protocol import get_credential_record, get_credential_records, create_credential_offer,accept_credential_offer,issue_credential

def get_invitation_str(connection):
    parts = connection.invitation.invitation_url.split("=")
    return parts[1]

## Issue Credential

Issuing a credential involves establishing a connection between the issuer and the holder, which is done by following the process outlined in "Example 01". Once the connection is established, the issuer will prepare and send a credential offer, which creates an issue record on both the issuer's and holder's agents. The holder will then retrieve the list of issue records, find the one they wish to accept, and notify the issuer of their acceptance. Finally, the issuer will issue the credential to the holder, completing the process.

## Client instances

We will create two separate clients, one for the issuer and one for the holder, in order to establish a connection between the two.

note: remember to update the file variables.env with the URLs and API keys provided to you.


In [2]:
load_dotenv("../BetaProgram/variables.env")
issuerApiKey = os.getenv('ISSUER_APIKEY')
issuerUrl = os.getenv('ISSUER_URL')

holderApiKey = os.getenv('HOLDER_APIKEY')
holderUrl = os.getenv('HOLDER_URL')

issuer_client = Client(base_url=issuerUrl, headers={"apiKey": issuerApiKey})
holder_client = Client(base_url=holderUrl, headers={"apiKey": holderApiKey})

### Create connection

For details on this see "Example 01 - Connections"

In [4]:
conn_request = CreateConnectionRequest()
conn_request.label = f'Issue credential {datetime.date.today().strftime("%Y-%m-%d %H:%M:%S")}'
issuer_connection: Response[Connection] =  create_connection.sync(client=issuer_client,json_body=conn_request)

invitation = get_invitation_str(issuer_connection)

accept_conn_request = AcceptConnectionInvitationRequest(invitation)
holder_connection: Response[ConnectionInvitation] =  accept_connection_invitation.sync(client=holder_client,json_body=accept_conn_request)


issuer_connection: Response[Connection] = get_connection.sync(client=issuer_client,connection_id=issuer_connection.connection_id)
holder_connection: Response[Connection] = get_connection.sync(client=holder_client,connection_id=holder_connection.connection_id)

while (issuer_connection.state != 'ConnectionResponseSent' and holder_connection.state != 'ConnectionResponseReceived'):
    issuer_connection: Response[Connection] = get_connection.sync(client=issuer_client,connection_id=issuer_connection.connection_id)
    holder_connection: Response[Connection] = get_connection.sync(client=holder_client,connection_id=holder_connection.connection_id)
    print("Issuer State: {} Holder State: {} \n".format(issuer_connection.state,holder_connection.state))
    time.sleep(1)
    
print("Connection established between Issuer and Holder!")
print(f"Issuer connection id: {issuer_connection.connection_id}")

Issuer State: InvitationGenerated Holder State: ConnectionRequestPending 

Issuer State: InvitationGenerated Holder State: ConnectionRequestPending 

Issuer State: ConnectionResponsePending Holder State: ConnectionRequestPending 

Issuer State: ConnectionResponsePending Holder State: ConnectionRequestSent 

Issuer State: ConnectionResponseSent Holder State: ConnectionRequestSent 

Connection established between Issuer and Holder!
Issuer connection id: 15ecfd56-c4fc-4c01-bfef-a8c02dd9dfdd


Flow 2 - Issue Credential
-------------------------

Issuer - create an invitation

Holder - accept invitation

Issuer - check connection
Issuer - credential offer

Holder - check records
Holder - accept offer

Issuer - check records
Issuer - issue credential

Holder - check record, credential

